# Load and preprocess data

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, RobertaForSequenceClassification, AdamW
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
# Preprocess data function which including lemmatized and POS
def genLemmatizedEssay(Lemmatized_Tokens, POS_Tags):
    tmp = map(lambda x: str(x[0]) + '_' + str(x[1][1]), zip(Lemmatized_Tokens, POS_Tags))
    return ' '.join(tmp)

def preProcess(filePath):
    # Step 1: Read the .txt file into a pandas DataFrame
    df = pd.read_csv(filePath, usecols=['Essay', 'Score'])

    # Step 2: Tokenization
    df['Tokens'] = df['Essay'].apply(word_tokenize)

    # Step 3: Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['Lemmatized_Tokens'] = df['Tokens'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

    # Step 4: Part-of-speech (POS) tagging
    df['POS_Tags'] = df['Lemmatized_Tokens'].apply(nltk.pos_tag)

    # Step 5: Join all word in lemmatized tokens
    df['Lemmatized_Essays'] = df.apply(lambda row: genLemmatizedEssay(row['Lemmatized_Tokens'], row['POS_Tags']), axis=1)

    return df

In [ ]:
filePath = '../Baseline_Filtered_Synthesized_Essays.csv'

df_processed = preProcess(filePath)

print(df_processed['Lemmatized_Essays'][:5])

# Import BERT model and create dataset for training

In [ ]:
# Load the pre-trained BERT model and modify the final layer for classification
bert_model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
)

# Set the optimizer and learning rate scheduler
optimizer = AdamW(bert_model.parameters(), lr=2e-5, eps=1e-8)

epochs = 2

In [ ]:
from torch.utils.data import Dataset, DataLoader

class POSDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label)
        }

# Fine-tune BERT model

In [ ]:
from transformers import RobertaTokenizer
bert_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
max_length = 128  # Maximum sequence length

In [ ]:
from sklearn.model_selection import KFold

# Define the number of folds for cross-validation
n_folds = 5

device = 'cpu'

# Initialize the cross-validator
kf = KFold(n_splits=n_folds, shuffle=True)

df_test = preProcess('../Data_Augment_Origin_2.csv')

test_dataset = POSDataset(df_test['Lemmatized_Essays'].to_list(), df_test['Score'].to_list(), bert_tokenizer, max_length)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Iterate over the folds
for fold, (train_indices, val_indices) in enumerate(kf.split(df_processed['Essay'])):
    print(f'Fold {fold+1}')
    print('length of train indices: ', len(train_indices))

    # Initialize the data loaders for training and validation
    df_train = df_processed.loc[train_indices]
    df_val = df_processed.loc[val_indices]

    train_dataset = POSDataset(df_train['Lemmatized_Essays'].to_list(), df_train['Score'].to_list(), bert_tokenizer, max_length)
    val_dataset = POSDataset(df_val['Lemmatized_Essays'].to_list(), df_val['Score'].to_list(), bert_tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    # Train and evaluate the model for each epoch
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}')
        
        # Train the model on the training data
        bert_model.train()
        train_loss = 0.0
        # print('train_loader: ', train_loader[0])
        for batch in train_loader:
            optimizer.zero_grad()
            outputs = bert_model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        print(f'Training Loss: {train_loss}')
        
        # Evaluate the model on the validation data
        bert_model.eval()
        val_loss = 0.0
        val_acc = 0.0
        with torch.no_grad():
            for batch in val_loader:
                outputs = bert_model(**batch)
                loss = outputs.loss
                logits = outputs.logits
                val_loss += loss.item()
                preds = logits.argmax(dim=1)
                val_acc += (preds == batch['labels']).float().mean().item()
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)
        print(f'Validation Loss: {val_loss}')
        print(f'Validation Accuracy: {val_acc}')

        # Test on original data
        bert_model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for batch in test_loader:
                outputs = bert_model(**batch)
                loss = outputs.loss
                logits = outputs.logits
                test_loss += loss.item()
                preds = logits.argmax(dim=1)
                test_acc += (preds == batch['labels']).float().mean().item()
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Test Loss: {test_loss}')
        print(f'Test Accuracy: {test_acc}')

In [ ]:
# Test on original data
import numpy as np
final_test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
bert_model.eval()
test_loss = 0.0
test_acc = 0.0
predictions = []
with torch.no_grad():
    for batch in final_test_loader:
        outputs = bert_model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        test_loss += loss.item()
        preds = logits.argmax(dim=1)
        predictions = np.append(np.array(predictions), np.array(preds.numpy()))
        print('Model predictions: ', predictions)
        print('Real labels: ', batch['labels'].numpy())
        test_acc += (preds == batch['labels']).float().mean().item()
test_loss /= len(final_test_loader)
test_acc /= len(final_test_loader)
df_test['Predictions'] = predictions
df_test.to_csv('Master_data_with_prediction_2_1_RoBERTa.csv', index=False)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score

# Step 1: Load the CSV file and separate the "score" and "prediction" columns
df = pd.read_csv("Master_data_with_prediction_2_1_RoBERTa.csv")
y_true = df["Score"]
y_pred = df["Predictions"]

# Step 2: Calculate accuracy for each score level
score_levels = sorted(df["Score"].unique())
accuracy_per_level = {}
for level in score_levels:
    mask = y_true == level
    accuracy_per_level[level] = accuracy_score(y_true[mask], y_pred[mask])

print("Accuracy for each score level:")
for level, accuracy in accuracy_per_level.items():
    print(f"Score {level}: {accuracy:.2f}")

print("Accuracy for all score level:")
print(f"Score: {accuracy_score(y_true, y_pred):.2f}")

# Step 3 and Step 4: Calculate the confusion matrix and QWK
conf_matrix = confusion_matrix(y_true, y_pred)
qwk= cohen_kappa_score(y_true, y_pred, weights="quadratic")

print("\nQWK:")
print(f"Score: {qwk:.2f}")